# Введение в искусственные нейронные сети
# Урок 7. Дектирование объектов

## Практическое задание

<ol>
    <li>Сделайте краткий обзор какой-нибудь научной работы посвященной тому или иному алгоритму для object detection, который не рассматривался на уроке. Проведите анализ: Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур? В чем плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении данной архитектуры на практике? 
    </li>
    <li>* Ссылка на репозиторий с полным кодом для обучения ssd нейросети - https://github.com/sergeyveneckiy/ssd-tensorflow. Попробуйте улучшить точность ее работы и напишите отчет, что вы пробовали изменить в ее параметрах и как это отражалось на процессе обучения нейронной сети. 
        Обратите внимание! Мин. сист. требования для запуска данного проекта - это минимум 8 Gb ОЗУ. Если у вас недостаточно мощности компьютера, то вы можете просто изучить содержимое исходного кода и датасета данного проекта.</li>

</ol>

##  Real-Time Multi-Class Scene Understanding
##  for Autonomous Driving Using Multiple Views

#### Ke Chen, Ryan Oldja, Nikolai Smolyanskiy, Stan Birchfield
#### Alexander Popov, David Wehr, Ibrahim Eden, Joachim Pehserl
#### NVIDIA
https://arxiv.org/pdf/2006.05518.pdf

Автономное вождение требует восприятия актуальной в этот момент информации. Требуется в реальном моменте времени распознавать полосы движения, искривления дороги, сигнал светофора, препятствия, расстояние до автомобилей спереди, пешеходов и т.д.
Исследователи предлагают методы использующие данные RGB картинки, данные лидара или их микс. Хотя как RGB распознавание довольно зрелая технология, она вносит заметные геометрические неточности. LIDAR решает эту проблему, но приносит другую - прямая обработка облака из 3D точек затруднительна. Один возможный путь здесь - взять 3D облако точек как воксельную сетку на вход DNN, но это все равно вычислительно дорого и имеет ошибки, другая проектировать это облако на вид птичьего глаза (Birds Eye View, BEV) и приментить 2D свертку, но здесь можно потерять сравнительно ценную информацию, например пешеходов из-за их малых размеров. В PIXOR используется 2D свертка на основе данных лидара, и проектируется на BEV тензор.  DNN вычисляет карты доверия объектов и регрессирует рамки объектов, размеры и ориентацию для каждого пикселя. Кластеризация используется для извлечения финальных позиций рамок. Такой метод довольно быстрый но может направильно определять объекты. Также VoxelNet , MV3D, PointNet, RangeNet++.

В этой статье авторы пытаются обойти эти ограничения. Для облегчения распознавания пешеходов используется семантическая сегментация точек лидара. Применяется двухступенчатая многоплановая сеть, которая выполняет семантическую сегментацию проекции вида, а затем используется определение объектов через BEV. Используется 2D свертка, которая значительно быстрее 3D.
На вход системы подается облако точек с обзором 360 град, которая проецируется сферически и сверху вниз(ортографическая проекция). На первом этапе извлекавется семантическая информация, которая делит сегменты изображения на классы: легковые автомобили, грузовики, пешеходы, велосипеды, дорогу, обочину и прочее.

![lidar1.png](attachment:lidar1.png)


Арахитектура похожа на FPN(feature pyramid network) по дизайну. Кодировщик состоит из 3 сверточных слоев с 64, 64 и 128 3х3 фильтрами, за ним следует три блока восприятия. Декодер возращает признаки назад к оригинальному разрешению через 3 блока развертки. За каждым блоком развертки стоит два сверточных слоя. Классификатор состоит из 3х3 сверточного слоя с размером 64 признака, затем 1х1 слой, который выдает 7-элементый вектор для пикселя. Каждый сверточный слой сопровождается батч нормализацией и ReLU активацией. 
Семантическ распознанный скан репроецируется на ортографическую проекцию. Вторая стадия архитектуры также шифратор-дешифратор неполносвязные с двумя центрами для классификации и регрессии рамок объектов. Используя вероятности классов(лучше чем сами классы), заставляет сеть выполнить более сложную и точную обработку (например пешеход или велосипед). Финальный выход состоит из массива 256х256 содержащего распределение классов вместе с другим массивом 256х256, содержащим расстояние до центра объекта, размеры и ориентацию. Этот этап натренирован с потерями, содержащими как фокальные потери, так и L1.  Кластеризующий алгоритм DBSCAN применяет на вход второй этап для определения объектов.
Итого, представлена новая мультиклассовая система мгновенного определения авто и пешеходов по данным лидара. Благодаря своему простому подходу система работает быстрее другие подходов: 150 fps  на встроенном GPU.
Авторы говорят о простоте подхода, объединяющим лучшее из разных подходов. Оба этапа могут обучаться независимо, например на датасетах SemanticKITTI и KITTI. Получена модель, способная достичь соревновательных результатов. Использовался Adam optimizer c lr= 0.0001. Обе стадии тренировались 40-50 эпох с батчем =4. 
В таблице приведены результаты сравнения с другими моделями, и получается что при сравнимой точности он будет быстрее. 


In [4]:
![lidar2.png](attachment:lidar2.png)

"[lidar2.png]" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [3]:
![lidar4.jpg](attachment:lidar4.jpg)

"[lidar4.jpg]" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [ ]:
![lidar3.png](attachment:lidar3.png)